In [1]:
import findspark
findspark.init("/opt/spark/")
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import regexp_replace
import configparser
import os
from pyspark.sql import functions as F
from pyspark.sql.functions import month, date_format
from pyspark.sql.functions import explode, col, from_json, struct
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import split, explode, regexp_replace
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

from pyspark.sql.functions import split, explode, regexp_replace, from_json

from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [2]:
spark = SparkSession.builder \
.appName("week10") \
.config("spark.jars.packages", "org.postgresql:postgresql:42.7.2") \
.master("local[2]") \
.getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/train/.ivy2/cache
The jars for the packages stored in: /home/train/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fdcdcd86-d7df-489c-ac81-84371206590d;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.2 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 329ms :: artifacts dl 8ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

In [3]:
! java --version


openjdk 11.0.22 2024-01-16 LTS
OpenJDK Runtime Environment (Red_Hat-11.0.22.0.7-1.el7_9) (build 11.0.22+7-LTS)
OpenJDK 64-Bit Server VM (Red_Hat-11.0.22.0.7-1.el7_9) (build 11.0.22+7-LTS, mixed mode, sharing)


In [4]:
df = spark.read \
.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.option("quote", "\"") \
.option("escape", "\"") \
.option("sep",",") \
.load("datasets/tmdb_5000_movies_and_credits/tmdb_5000_credits.csv")


In [5]:
df.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|               title|                cast|                crew|
+--------+--------------------+--------------------+--------------------+
|   19995|              Avatar|[{"cast_id": 242,...|[{"credit_id": "5...|
|     285|Pirates of the Ca...|[{"cast_id": 4, "...|[{"credit_id": "5...|
|  206647|             Spectre|[{"cast_id": 1, "...|[{"credit_id": "5...|
|   49026|The Dark Knight R...|[{"cast_id": 2, "...|[{"credit_id": "5...|
|   49529|         John Carter|[{"cast_id": 5, "...|[{"credit_id": "5...|
|     559|        Spider-Man 3|[{"cast_id": 30, ...|[{"credit_id": "5...|
|   38757|             Tangled|[{"cast_id": 34, ...|[{"credit_id": "5...|
|   99861|Avengers: Age of ...|[{"cast_id": 76, ...|[{"credit_id": "5...|
|     767|Harry Potter and ...|[{"cast_id": 3, "...|[{"credit_id": "5...|
|  209112|Batman v Superman...|[{"cast_id": 18, ...|[{"credit_id": "5...|
|    1452|    Superman Returns|[{"cast

In [6]:
df.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- crew: string (nullable = true)



In [9]:
#şema
crew_schema = StructType([
    StructField("credit_id", StringType(), True),
    StructField("department", StringType(), True),
    StructField("gender", IntegerType(), True),
    StructField("id", IntegerType(), True),
    StructField("job", StringType(), True),
    StructField("name", StringType(), True)
])

# crew sutunu JSON 
df = df.withColumn("crew", from_json(col("crew"), ArrayType(crew_schema)))
# crew sutunu explode 
df = df.withColumn("crew", explode(col("crew")))


# credits_crew df
credits_crew = df.select(
    col("movie_id"),
    col("title"),
    col("crew.credit_id"),
    col("crew.department"),
    col("crew.gender"),
    col("crew.id"),
    col("crew.job"),
    col("crew.name")
)

#show
credits_crew.show(truncate=False)




+--------+------+------------------------+-----------------+------+----+------------------------+------------------+
|movie_id|title |credit_id               |department       |gender|id  |job                     |name              |
+--------+------+------------------------+-----------------+------+----+------------------------+------------------+
|19995   |Avatar|52fe48009251416c750aca23|Editing          |0     |1721|Editor                  |Stephen E. Rivkin |
|19995   |Avatar|539c47ecc3a36810e3001f87|Art              |2     |496 |Production Design       |Rick Carter       |
|19995   |Avatar|54491c89c3a3680fb4001cf7|Sound            |0     |900 |Sound Designer          |Christopher Boyes |
|19995   |Avatar|54491cb70e0a267480001bd0|Sound            |0     |900 |Supervising Sound Editor|Christopher Boyes |
|19995   |Avatar|539c4a4cc3a36810c9002101|Production       |1     |1262|Casting                 |Mali Finn         |
|19995   |Avatar|5544ee3b925141499f0008fc|Sound            |2   

In [8]:

#sadece from_json yapılmıs hali
from pyspark.sql.functions import col, explode
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Şema
crew_schema = StructType([
    StructField("credit_id", StringType(), True),
    StructField("department", StringType(), True),
    StructField("gender", IntegerType(), True),
    StructField("id", IntegerType(), True),
    StructField("job", StringType(), True),
    StructField("name", StringType(), True)
])

# crew sütununu JSON 
crew_json = df.withColumn("crew", F.from_json(F.col("crew"), ArrayType( crew_schema )))

# 
credits_crew2 = crew_json.select(
    "movie_id",
    "title",
    "crew.credit_id",
    "crew.department",
    "crew.gender",
    "crew.id",
    "crew.job",
    "crew.name"
)

#
credits_crew2.show()


+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|               title|           credit_id|          department|              gender|                  id|                 job|                name|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   19995|              Avatar|[52fe48009251416c...|[Editing, Art, So...|[0, 2, 0, 0, 1, 2...|[1721, 496, 900, ...|[Editor, Producti...|[Stephen E. Rivki...|
|     285|Pirates of the Ca...|[52fe4232c3a36847...|[Camera, Directin...|[2, 2, 2, 2, 2, 0...|[120, 1704, 770, ...|[Director of Phot...|[Dariusz Wolski, ...|
|  206647|             Spectre|[54805967c3a36829...|[Sound, Directing...|[2, 2, 1, 2, 2, 2...|[153, 39, 8384, 9...|[Original Music C...|[Thomas Newman, S...|
|   49026|The Dark Knight R...|[52fe4781c3a36847...|